In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptF = pd.DataFrame(botpt_data)
df_botptF['bottom_pressure'] = df_botptF['bottom_pressure'].astype(float)
df_botptF['depth']=df_botptF['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptF.index.to_pydatetime()]
df_botptF['epoch'] = epoch
df_botptF= df_botptF.sort_index()

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_E11.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE= df_botptE.sort_index()

In [ ]:
df_botptF.describe()

In [ ]:
df_botptE.head()

In [ ]:
df_botptF.tail()

In [ ]:
df_botptE.tail()

In [ ]:
del df_botptE['epoch']
del df_botptE['bottom_pressure']
del df_botptF['epoch']
del df_botptF['bottom_pressure']

In [ ]:
x = df_botptE.head()
y = df_botptF.head()

In [ ]:
z = pd.merge(x,y,how='outer', indicator=True, left_index=True, right_index=True)

In [ ]:
test = pd.merge(df_botptF, df_botptE,how='outer', indicator=True, left_index=True, right_index=True, suffixes=('_F', '_E'))

In [ ]:
df_botptMerge = test[test['_merge'] == 'both']
del df_botptMerge['_merge']

In [ ]:
df_botptMerge = df_botptMerge.loc['2017-01-1 00:00:00':'2018-10-31 00:00:00']

#### Calculate Depth difference 

In [ ]:
depthDiff = df_botptMerge['depth_E'].values - df_botptMerge['depth_F'].values

In [ ]:
df_botptMerge['diff'] = depthDiff

In [ ]:
h2 = df_botptMerge['depth_F'].tolist()
h1 = df_botptMerge['depth_E'].tolist()

In [ ]:
def subtract_lists(h1, h2):
    diff = [a - b for a,b in zip(h1, h2)]
    return diff

In [ ]:
height = subtract_lists(h1,h2)

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
#height = df_botptMerge['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMerge.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))
df_botptMerge.head()

#### Import Seismic Data 

In [ ]:
seismic_file = '/home/jovyan/data/hypo71_2018.dat'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2018-10-31 00:00:00']
del df_seismic_data['yyyymmdd']
del df_seismic_data['HHMM']
del df_seismic_data['SSS.SS']
#del df_seismic_data['MW']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']
df_seismic_data['MW']= df_seismic_data['MW'].astype('float64').values
df_seismic_data['Depth']= df_seismic_data['Depth'].astype('float64').values
df_seismic_data['RMS']= df_seismic_data['RMS'].astype('float64').values
df_seismic_data.head()

In [ ]:
df_seismic_data.plot(x='datetime', y= 'Depth')

In [ ]:
df_seismic_data.plot(x='datetime', y= 'MW')

In [ ]:
df_seismic_data.plot(x='datetime', y= 'RMS')

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
df_seismic_data.head()

#### Create dataframe with Earthquake frequency (count per day)
##### Note: Days with zero earthquakes are not represented in this timeseries. 

In [ ]:
df_eqMean=df_seismic_data.groupby('date').mean()
df_eqCount= df_seismic_data.groupby('date').count()
#del df_eqMean['datetime']
#df_eqMean.columns.name = df_eqMean.index.name
#df_eqMean.index.name = None
df_eqMean.head()

In [ ]:
df_eqCount['count'] = df_eqCount.datetime.astype('float64').values
df_eqCount.head()

In [ ]:
df_eqMean['count'] = df_eqCount['count'].values
df_eqMean.head()

#### Resample. Add days to our earthquake data where there were zero earthquakes. 

In [ ]:
idx = pd.date_range('2017-01-1 00:00:00', '2018-10-31 00:00:00')

df_eqMean = df_eqMean

df_eqMean.index = pd.DatetimeIndex(df_eqMean.index)

df_eqMean = df_eqMean.reindex(idx, fill_value=0)
df_eqMean.head()

In [ ]:
import hvplot.pandas
a= df_eqMean.hvplot.scatter(y= 'count')
b= df_eqMean.hvplot.scatter(y= 'Depth')
a+b

In [ ]:
df_eqMean.hvplot.scatter(y= ['count', 'Depth', 'RMS'], 
                         subplots=True, shared_axes=False).cols(1)

In [ ]:
df_botptMerge.hvplot()

In [ ]:
# df_botptMerge.hvplot(x='date', y='diff', datashade=True, height=200,
#                        flip_yaxis=True)